In [0]:
pip install gensim

Python interpreter will be restarted.
Requirement already satisfied: gensim in /databricks/python3/lib/python3.8/site-packages (4.2.0)
Requirement already satisfied: scipy>=0.18.1 in /databricks/python3/lib/python3.8/site-packages (from gensim) (1.6.2)
Requirement already satisfied: smart-open>=1.8.1 in /databricks/python3/lib/python3.8/site-packages (from gensim) (6.0.0)
Requirement already satisfied: numpy>=1.17.0 in /databricks/python3/lib/python3.8/site-packages (from gensim) (1.19.2)
Python interpreter will be restarted.

In [0]:
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
import pandas as pd
import pickle

ICD9CODES = pickle.load(open("/dbfs/FileStore/data_2/ICD9CODES.p", 'rb'))
ICD9CODES_TOP10 = pickle.load(open("/dbfs/FileStore/data_2/ICD9CODES_TOP10.p", 'rb'))
ICD9CODES_TOP50 = pickle.load(open("/dbfs/FileStore/data_2/ICD9CODES_TOP50.p", 'rb'))
ICD9CAT_TOP10 = pickle.load(open("/dbfs/FileStore/data_2/ICD9CAT_TOP10.p", 'rb'))
ICD9CAT_TOP50 = pickle.load(open("/dbfs/FileStore/data_2/ICD9CAT_TOP50.p", 'rb'))

In [0]:

df1_sp = spark.sql('''
select distinct `row_id` as id,
`0389`,
`2449`,
`25000`,
`2720`,
`2724`,
`2760`,
`2761`,
`2762`,
`2767`,
`2851`,
`2859`,
`2875`,
`3051`,
`311`,
`32723`,
`4019`,
`40390`,
`40391`,
`41071`,
`412`,
`41401`,
`4168`,
`4240`,
`4241`,
`42731`,
`42789`,
`4280`,
`486`,
`49390`,
`496`,
`5070`,
`5119`,
`5180`,
`51881`,
`53081`,
`5845`,
`5849`,
`5859`,
`5990`,
`7742`,
`78552`,
`99592`,
`9971`,
`V053`,
`V1582`,
`V290`,
`V4581`,
`V4582`,
`V5861`,
`V5867`,
`text`
--`clean_text`
from tempdb.hadm_top_50_icd_step_5
''')

df1_pd = df1_sp.toPandas()


In [0]:
df_hadm_top10 = df1_pd[['id','4019','2724','25000','4280','41401','53081','51881','42731','5849','5990','text']]

In [0]:
import random

def separate(seed, N):    
    idx=list(range(N))
    random.seed(seed)
    random.shuffle(idx)
    idx_train= idx[0:int(N*0.50)]
    idx_val= idx[int(N*0.50):int(N*0.75)]
    idx_test= idx[int(N*0.75):N]

    return idx_train, idx_val, idx_test


idx_train, idx_val, idx_test = separate(1234, df_hadm_top10.shape[0])
idx_join_train=idx_train + idx_val
len(idx_join_train)

Out[4]: 1363464

In [0]:
df_hadm_top10_d2v=df_hadm_top10.iloc[idx_join_train].copy()

In [0]:
# Cleanning the data
# Light preprocesing done on purpose (so word2vec understand sentence structure)
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', ' ', text.lower())
    text = text.split()
    return text

token_review = list(df_hadm_top10_d2v['text'].apply(preprocessor))
len(token_review)

Out[15]: 1363464

In [0]:
token_review

Out[16]: [['2129',
 '5',
 '18',
 '2',
 '00',
 'pm',
 'babygram',
 'chest',
 'only',
 'port',
 'clip',
 'clip',
 'number',
 'radiology',
 '63972',
 'reason',
 'prematurity',
 'to',
 'rule',
 'out',
 'rds',
 '______________________________________________________________________________',
 'hospital',
 '4',
 'medical',
 'condition',
 'infant',
 'with',
 'prematurity',
 'and',
 'respiratory',
 'distress',
 'reason',
 'for',
 'this',
 'examination',
 'prematurity',
 'to',
 'rule',
 'out',
 'rds',
 '______________________________________________________________________________',
 'final',
 'report',
 'indications',
 'chest',
 'this',
 'is',
 'a',
 'twin',
 'born',
 'prematurely',
 'films',
 'today',
 'demonstrate',
 'low',
 'lung',
 'volumes',
 'with',
 'increased',
 'opacity',
 'throughout',
 'both',
 'lungs',
 'compatible',
 'with',
 'hilan',
 'membrane',
 'disease',
 'no',
 'other',
 'abnormalities',
 'are',
 'apparent'],
 ['2146',
 '1',
 '17',
 '10',
 '41',
 'am',
 'ct',
 'abdomen',
 'w',
 'o',
 'contrast',
 'ct',
 'pelvis',
 'w',
 'o',
 'contrast',
 'clip',
 'clip',
 'number',
 'radiology',
 '94220',
 'reason',
 'acute',
 'abd',
 'pain',
 'hypotension',
 'eval',
 'for',
 'abd',
 'process',
 'free',
 'air',
 'field',
 'of',
 'view',
 '38',
 '______________________________________________________________________________',
 'hospital',
 '2',
 'medical',
 'condition',
 '85',
 'year',
 'old',
 'woman',
 'with',
 'reason',
 'for',
 'this',
 'examination',
 'acute',
 'abd',
 'pain',
 'hypotension',
 'eval',
 'for',
 'abd',
 'process',
 'free',
 'air',
 'no',
 'contraindications',
 'for',
 'iv',
 'contrast',
 '______________________________________________________________________________',
 'wet',
 'read',
 'enya',
 'mon',
 '2146',
 '1',
 '17',
 '12',
 '27',
 'pm',
 'pfi',
 '1',
 'significant',
 'bibasilar',
 'pleural',
 'effusion',
 'simple',
 'fluid',
 'attentuation',
 'left',
 'greater',
 'than',
 'right',
 'lobar',
 'collapse',
 'on',
 'the',
 'left',
 'and',
 'segmental',
 'collapse',
 'on',
 'the',
 'right',
 'passive',
 'consolidation',
 '2',
 'interval',
 'decrease',
 'of',
 'right',
 'rectus',
 'sheath',
 'hematoma',
 '3',
 'uncomplicated',
 'cholelithiasis',
 '4',
 'diffuse',
 'diverticular',
 'disease',
 'without',
 'evidence',
 'of',
 'diverticulitis',
 'thickening',
 'of',
 'rectal',
 'wall',
 'please',
 'correlate',
 'if',
 'history',
 'of',
 'prolapse',
 '5',
 'atrophic',
 'kidneys',
 'and',
 'moderate',
 'vascular',
 'calcification',
 'unchanged',
 '______________________________________________________________________________',
 'final',
 'report',
 'history',
 '85',
 'year',
 'old',
 'female',
 'with',
 'history',
 'of',
 'heart',
 'attack',
 'now',
 'presents',
 'with',
 'acute',
 'abdominal',
 'pain',
 'hypotension',
 'evaluate',
 'for',
 'abdominal',
 'process',
 'question',
 'of',
 'free',
 'air',
 'study',
 'ct',
 'abdomen',
 'and',
 'pelvis',
 'without',
 'iv',
 'contrast',
 'multiplanar',
 'reformatted',
 'images',
 'were',
 'obtained',
 'comparison',
 'ct',
 'abdomen',
 'and',
 'pelvis',
 'on',
 '2145',
 '5',
 '4',
 'findings',
 'ct',
 'abdomen',
 'without',
 'contrast',
 'there',
 'is',
 'significant',
 'bibasilar',
 'pleural',
 'effusion',
 'with',
 'simple',
 'fluid',
 'attenuation',
 'left',
 'greater',
 'than',
 'right',
 'there',
 'is',
 'left',
 'sided',
 'lobar',
 'collapse',
 'and',
 'right',
 'sided',
 'segmental',
 'collapse',
 'with',
 'passive',
 'consolidation',
 'there',
 'is',
 'mild',
 'to',
 'moderate',
 'coronary',
 'calcifications',
 'the',
 'cardiac',
 'size',
 'is',
 'within',
 'normal',
 'limits',
 'there',
 'is',
 'a',
 'central',
 'line',
 'visualized',
 'in',
 'the',
 'liver',
 'there',
 'is',
 'no',
 'focal',
 'lesion',
 'allowing',
 'the',
 'limits',
 'of',
 'a',
 'noncontrast',
 'study',
 'there',
 'is',
 'evidence',
 'of',
 'a',
 'small',
 'gallstone',
 'with',
 'sludge',
 'but',
 'no',
 'evidence',
 'of',
 'biliary',
 'obstruction',
 'or',
 'acute',
 'inflammation',
 'the',
 'pa

In [0]:
from gensim.models import Word2Vec
#import gensim.models.Word2Vec
from gensim import utils
from time import time

# assumptions: window is 5 words left and right, eliminate words than dont occur in
# more than 10 docs, use 4 workers for a quadcore machine. Size is the size of vector
# negative=5 implies negative sampling and makes doc2vec faster to train
# sg=0 means CBOW architecture used. sg=1 means skip-gram is used
#model = Doc2Vec(sentence, size=100, window=5, workers=4, min_count=5)


import random

In [0]:
import gensim

In [0]:
model_w2v=gensim.models.Word2Vec.load('/dbfs/FileStore/data/model_word2vec_skipgram_300dim')

In [0]:

size = 300  #change to 100 and 600 to generate vectors with those dimensions

#instantiate our  model
model_w2v = Word2Vec(min_count=10, window=5, vector_size=size, sample=1e-3, negative=5, workers=4, sg=0)

#build vocab over all reviews
model_w2v.build_vocab(token_review)

#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.
Idx=list(range(len(token_review)))

t0 = time()
random.shuffle(Idx)
perm_sentences = [token_review[i] for i in Idx]
model_w2v.train(perm_sentences,total_examples = len(token_review),epochs = 5)
# for epoch in range(5):

#      print(epoch)
    
elapsed=time() - t0
print("Time taken for Word2vec training: ", elapsed, "seconds.")

Time taken for Word2vec training: 1207.28742146492 seconds.

In [0]:
model_w2v.wv

Out[20]: <gensim.models.keyedvectors.KeyedVectors at 0x7f7f55ae8580>

In [0]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!

In [0]:
stopwords.words('english')

Out[22]: ['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don't",
 'should',
 "should've",
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 'aren',
 "aren't",
 'couldn',
 "couldn't",
 'didn',
 "didn't",
 'doesn',
 "doesn't",
 'hadn',
 "hadn't",
 'hasn',
 "hasn't",
 'haven',
 "haven't",
 'isn',
 "isn't",
 'ma',
 'mightn',
 "mightn't",
 'mustn',
 "mustn't",
 'needn',
 "needn't",
 'shan',
 "shan't",
 'shouldn',
 "shouldn't",
 'wasn',
 "wasn't",
 'weren',
 "weren't",
 'won',
 "won't",
 'wouldn',
 "wouldn't"]

In [0]:
model_w2v.wv.save_word2vec_format("/dbfs/FileStore/data/word2vec.txt", binary=False)

In [0]:
model_w2v.wv.vector_size

Out[30]: 300

In [0]:
# import numpy as np  # Make sure that numpy is imported
# from nltk.corpus import stopwords

STOPWORDS_WORD2VEC = stopwords.words('english') + ICD9CODES

keys_updated = [word for word in list(model_w2v.wv.index_to_key) if word not in STOPWORDS_WORD2VEC]
index2word_set=set(keys_updated)

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    #index2word_set = set(model.wv.index2word) #activate if using gensim

    # activate if uploaded text version
    #index2word_set=set(keys_updated)
    
    
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model.wv[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec

In [0]:
def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 10000th review
       if counter%10000 == 0:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [0]:
from time import time
import numpy as np
t0 = time()
final_w2v = getAvgFeatureVecs(token_review, model_w2v, num_features=300)

Review 0 of 1363464
Review 10000 of 1363464
<command-3162767139708315>:35: RuntimeWarning: invalid value encountered in true_divide
 featureVec = np.divide(featureVec,nwords)
Review 20000 of 1363464
Review 30000 of 1363464
Review 40000 of 1363464
Review 50000 of 1363464
Review 60000 of 1363464
Review 70000 of 1363464
Review 80000 of 1363464
Review 90000 of 1363464
Review 100000 of 1363464
Review 110000 of 1363464
Review 120000 of 1363464
Review 130000 of 1363464
Review 140000 of 1363464
Review 150000 of 1363464
Review 160000 of 1363464
Review 170000 of 1363464
Review 180000 of 1363464
Review 190000 of 1363464
Review 200000 of 1363464
Review 210000 of 1363464
Review 220000 of 1363464
Review 230000 of 1363464
Review 240000 of 1363464
Review 250000 of 1363464
Review 260000 of 1363464
Review 270000 of 1363464
Review 280000 of 1363464
Review 290000 of 1363464
Review 300000 of 1363464
Review 310000 of 1363464
Review 320000 of 1363464
Review 330000 of 1363464
Review 340000 of 1363464
Review 350000 of 1363464
Review 360000 of 1363464
Review 370000 of 1363464
Review 380000 of 1363464
Review 390000 of 1363464
Review 400000 of 1363464
Review 410000 of 1363464
Review 420000 of 1363464
Review 430000 of 1363464
Review 440000 of 1363464
Review 450000 of 1363464
Review 460000 of 1363464
Review 470000 of 1363464
Review 480000 of 1363464
Review 490000 of 1363464
Review 500000 of 1363464
Review 510000 of 1363464
Review 520000 of 1363464
Review 530000 of 1363464
Review 540000 of 1363464
Review 550000 of 1363464
Review 560000 of 1363464
Review 570000 of 1363464
Review 580000 of 1363464
Review 590000 of 1363464
Review 600000 of 1363464
Review 610000 of 1363464
Review 620000 of 1363464
Review 630000 of 1363464
Review 640000 of 1363464
Review 650000 of 1363464
Review 660000 of 1363464
Review 670000 of 1363464
Review 680000 of 1363464
Review 690000 of 1363464
Review 700000 of 1363464
Review 710000 of 1363464
Review 720000 of 1363464
Review 730000 of 1363464
Review 740000 of 1363464
Review 750000 of 1363464
Review 760000 of 1363464
Review 770000 of 1363464
Review 780000 of 1363464
Review 790000 of 1363464
Review 800000 of 1363464
Review 810000 of 1363464
Review 820000 of 1363464
Review 830000 of 1363464
Review 840000 of 1363464
Review 850000 of 1363464
Review 860000 of 1363464
Review 870000 of 1363464
Review 880000 of 1363464
Review 890000 of 1363464
Review 900000 of 1363464
Review 910000 of 1363464
Review 920000 of 1363464
Review 930000 of 1363464
Review 940000 of 1363464
Review 950000 of 1363464
Review 960000 of 1363464
Review 970000 of 1363464
Review 980000 of 1363464
Review 990000 of 1363464
Review 1000000 of 1363464
Review 1010000 of 1363464
Review 1020000 of 1363464
Review 1030000 of 1363464
Review 1040000 of 1363464
Review 1050000 of 1363464
Review 1060000 of 1363464
Review 1070000 of 1363464
Review 1080000 of 1363464
Review 1090000 of 1363464
Review 1100000 of 1363464
Review 1110000 of 1363464
Review 1120000 of 1363464
Review 1130000 of 1363464
Review 1140000 of 1363464
Review 1150000 of 1363464
Review 1160000 of 1363464
Review 1170000 of 1363464
Review 1180000 of 1363464
Review 1190000 of 1363464
Review 1200000 of 1363464
Review 1210000 of 1363464
Review 1220000 of 1363464
Review 1230000 of 1363464
Review 1240000 of 1363464
Review 1250000 of 1363464
Review 1260000 of 1363464
Review 1270000 of 1363464
Review 1280000 of 1363464
Review 1290000 of 1363464
Review 1300000 of 1363464
Review 1310000 of 1363464
Review 1320000 of 1363464
Review 1330000 of 1363464
Review 1340000 of 1363464
Review 1350000 of 1363464
Review 1360000 of 1363464

In [0]:
final_w2v_df.head()

Out[25]:

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.139967,0.033714,0.153439,0.445777,0.037970,-0.090719,0.226456,-0.195170,0.239328,0.333894,0.238507,0.119953,0.033705,0.255410,-0.188587,-0.016762,0.221521,-0.070051,0.200918,-0.243817,0.023793,-0.221970,0.288584,0.059937,-0.142988,-0.118355,0.147534,0.435644,-0.247696,-0.007521,0.179609,0.054069,-0.249536,-0.117459,0.106328,-0.139409,0.014925,-0.161995,0.132676,-0.015941,...,-0.129139,0.162655,-0.260157,-0.088296,0.106183,0.332124,0.052802,-0.009563,-0.204580,0.098839,-0.002479,-0.480822,0.411926,-0.470149,0.312282,-0.070067,-0.345056,-0.459075,0.208422,0.449574,0.203541,-0.184366,0.097823,0.284013,-0.071914,0.039146,0.070052,0.030371,0.247537,0.039411,-0.068914,0.277303,-0.037943,0.046379,0.216274,0.196249,0.387682,-0.261909,-0.104117,-0.046064
1,0.434469,-0.606517,0.422965,0.222804,0.321507,0.662187,-0.512804,0.749808,1.088438,0.127547,-0.284722,0.080795,0.438144,0.022504,-1.288477,1.093791,-0.292396,-0.628237,0.137580,-0.464446,-0.296002,-0.225448,0.989165,-0.131898,-0.488450,0.025855,-0.098801,0.039629,-0.447080,-1.873182,0.214178,-0.244132,-0.132334,-0.045305,-0.612650,0.069913,0.673896,0.779190,0.817446,-0.944459,...,-0.664076,-1.078631,-0.914931,0.768214,0.063678,-0.301553,-0.214686,0.002517,-0.327858,0.072580,0.536694,-1.210013,0.696390,-0.727745,0.413748,-0.071461,-0.979326,-0.326534,-0.847112,-0.985298,-0.598812,0.535478,-0.408164,0.254184,-0.423437,-0.657641,0.327275,-0.584568,-0.453070,0.419913,0.562185,-0.093050,-0.194352,-0.349503,0.387395,0.893376,-0.636773,-0.502745,-0.007373,0.073770
2,0.028160,-0.762154,0.118022,-0.044584,-0.764408,0.098286,-0.249926,0.295553,-0.010283,-0.409701,-0.337034,1.240618,-0.341363,-1.006934,-1.625044,1.051519,-0.619101,-0.503064,-0.093201,0.610304,-0.120513,0.482451,0.725329,0.071022,0.156752,0.829768,-1.097052,-0.174353,-0.160264,-0.623375,-1.067595,-0.476856,0.556261,-0.454429,0.432018,0.646805,0.189934,-0.575974,-0.486951,-0.031989,...,0.249475,-0.018849,-0.249748,0.267388,-1.038372,0.163399,-0.291223,0.137503,-0.198867,0.830410,0.485469,-0.034434,-0.086853,0.364317,-1.110410,0.764739,0.431301,0.363901,-0.452942,-0.734132,-0.489920,0.094475,0.419809,-0.403488,0.308949,0.018214,-0.097912,-0.139159,-0.173676,0.630775,0.176851,0.114941,0.970527,0.707011,0.385428,-0.303767,0.224311,0.500264,0.610500,1.196223
3,0.054624,-0.028088,0.316749,-0.020224,0.102353,-0.060221,0.136424,-0.533246,-0.588628,0.489727,0.035663,0.153789,-0.162237,0.003637,-0.238743,-0.189118,-0.176616,0.357875,0.085724,0.725736,0.725889,-0.191687,-0.013253,-0.779848,-0.027067,0.462467,-0.138833,0.198378,0.151846,0.173199,0.288016,-0.355025,0.249748,0.350224,0.076976,0.226195,-0.397408,-0.325358,-0.341191,0.277511,...,-0.454239,0.504601,0.271907,0.021839,0.151364,0.420637,-0.432054,-0.161417,0.046912,0.035481,-0.076406,-0.217527,0.090501,-0.171969,0.778488,-0.118063,0.419415,0.327816,0.425221,0.085077,-0.020444,-0.088860,0.362768,-0.036160,-0.098962,0.318880,-0.070748,-0.229800,0.072210,-0.133077,-0.096115,0.594838,-0.230638,0.473088,-0.010964,-0.417244,-0.301926,0.622977,-0.139085,-0.509338
4,0.144380,0.063484,0.375551,0.107345,0.329128,0.156181,-0.268919,-0.042206,1.076669,0.157297,1.174725,0.590928,0.911411,0.271510,-0.841455,0.091073,0.523035,0.660252,0.873908,-0.615521,-0.119146,-0.157037,-0.102512,0.250339,-1.210010,-0.592090,0.171515,0.904109,-0.832902,-0.848771,-0.385110,0.019482,-1.242580,-0.845995,-0.126744,-0.607230,0.506003,-0.107226,-0.515833,-0.116456,...,-0.198180,-0.393576,-0.540214,0.660757,0.654644,0.122871,1.925829,0.333160,0.348364,0.449423,-0.102007,-0.430097,0.233151,-0.579610,-0.016657,0.242289,-0.829026,-0.166252,0.058278,-0.325322,0.199901,-0.997539,1.115800,0.529825,-0.448102,0.655156,0.546337,0.716052,0.733898,-1.221

In [0]:
# Create train set and test set to use Machine Learning model
import random
import pandas as pd
final_w2v_df=pd.DataFrame(data=final_w2v)  

In [0]:
labels=["id","4019", "2724","25000","4280","41401","53081","51881","42731","5849","5990"]

In [0]:
data_final=pd.concat([df_hadm_top10[labels],final_w2v_df], axis=1)

old split logic, splitting done in preprocessing

In [0]:
idx=list(range(len(token_review)))
random.seed(1234)
random.shuffle(idx)

In [0]:
idx_train= idx[0:int(len(data_final)*0.50)]
idx_val= idx[int(len(data_final)*0.50):int(len(data_final)*0.75)]
idx_test= idx[int(len(data_final)*0.75):len(data_final)]
train_set = data_final.iloc[idx_train]
val_set = data_final.iloc[idx_val]
test_set = data_final.iloc[idx_test]

upload dataset back to a table to be used for modeling

In [0]:
data_final

In [0]:
s_data_final = spark.createDataFrame(data_final)

id,4019,2724,25000,4280,41401,53081,51881,42731,5849,5990,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
55719,1,0,0,0,0,0,0,0,0,0,-1.1525245,0.15723443,0.1838233,-0.57781225,0.3511713,1.0269778,1.1865035,-0.3645481,-0.32536992,0.7053559,-0.05729557,0.30402762,-0.511216,0.3449339,-0.26530087,-0.19545084,-0.035034392,0.6030845,0.32382196,0.058272216,1.1496192,-0.07583337,-0.38381022,-0.038313106,-0.059901193,-0.009235247,0.3108128,-0.16180591,0.076915234,-0.628948,0.012136114,0.34915993,0.11169649,-0.027603326,0.12088766,0.015754415,-0.07528986,0.7361764,-0.375096,0.6838765,0.34126616,-0.17232133,0.5670233,0.1842313,-0.9191954,-0.05715491,-0.39360154,0.82914567,0.7040055,-0.9378289,-0.6692771,0.20666184,0.4659079,-0.9852135,-0.05060219,-0.8569875,-0.38273558,-1.0934033,0.09667993,0.5391056,-0.21248612,0.97338355,-0.25036633,0.13185519,-1.0454012,0.70061225,0.56089413,-0.23353964,0.3873815,0.61252654,-0.18594839,-1.1697508,-0.13116485,-0.15274367,-0.3425766,0.7229913,0.46215582,0.19172013,0.7315542,0.20280124,-0.47283942,0.11268868,0.21760689,-1.2414601,-0.54571706,0.6389718,-0.16051203,0.20270866,0.3108413,-0.417417,0.5325184,-0.30982396,0.3395837,-0.054592233,0.18700255,0.020021215,-0.44557497,0.17568253,0.066207305,-0.41227284,0.038476147,-0.56490237,-0.23229218,0.3676868,1.0247589,0.45013517,0.45407248,0.15603545,-0.111387715,0.011789778,-0.33382058,0.28754377,-0.3872725,0.79699403,-0.07860645,-0.6673928,-0.052668147,-0.15879771,0.6256208,0.028558478,-0.30958888,-0.7050788,0.22046235,-0.78366685,0.098655045,0.50495523,0.4559019,0.17166498,-0.43103623,-0.54671085,-0.65522283,-1.557738,-0.76205677,-0.35500312,-0.11766226,0.14130858,0.10890559,0.13084802,0.4764247,0.0681049,0.19539425,0.09600227,-0.304814,-0.27403235,1.1285267,0.24745204,0.33300155,0.23377757,0.005707037,0.076807685,0.6401494,-0.3869688,-0.69027025,0.2385011,0.02566953,-0.46519974,-0.08308293,-0.3190682,-0.32580185,-0.34240428,0.23345542,0.06658649,-0.1667445,0.44380632,0.08797021,0.3555495,0.9894684,0.804572,0.7345692,0.33207488,-0.17618023,-0.6734484,-0.59454125,0.6396385,-0.0023660294,-0.8215612,0.28582826,0.49408203,-0.80100435,-0.2171121,-0.737516,0.16058105,-0.572672,0.67947215,1.0157514,-0.46116358,0.14001904,0.47349715,-0.5133047,0.45741102,-0.47147244,0.14892927,0.15213883,-0.1770583,-0.14278744,-0.7972758,-0.96667296,0.25489458,-0.5524288,-0.17040165,5.393658E-5,-0.13759252,-0.44481844,0.50669485,0.47381333,-0.51452607,-0.1677315,-0.25661966,-0.060210813,-0.41531605,-0.7907455,-1.0915484,0.3523149,0.11650098,-0.9119422,1.2233778,0.5198397,0.69896245,0.8568886,-0.4719389,0.221684,-0.120144926,-0.91353774,0.15424588,0.30815247,0.898391,-0.2668036,0.6571687,-0.5245165,-0.04805845,-0.4360594,0.45182943,-0.32383764,-0.2717177,-0.029871043,-0.3980963,0.24144305,-0.12073473,-0.3863538,-0.29163685,-0.81181383,0.2518887,-0.15020402,-1.2167256,0.09825281,0.25232646,0.35458043,-0.42285445,0.5985994,0.2958827,-0.12750606,-0.2917953,0.468063

In [0]:
model_w2v.save('/dbfs/FileStore/data/model_word2vec_skipgram_300dim.txt')

In [0]:
s_data_final.registerTempTable("word2vec_completeset")

/databricks/spark/python/pyspark/sql/dataframe.py:146: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
 warnings.warn(

In [0]:
%sql
create table tempdb.word2vec_completeset as (
select * 
from word2vec_completeset
)

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * 
from tempdb.word2vec_completeset
limit 10


id,4019,2724,25000,4280,41401,53081,51881,42731,5849,5990,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
55719,1,0,0,0,0,0,0,0,0,0,-1.1525245,0.15723443,0.1838233,-0.57781225,0.3511713,1.0269778,1.1865035,-0.3645481,-0.32536992,0.7053559,-0.05729557,0.30402762,-0.511216,0.3449339,-0.26530087,-0.19545084,-0.035034392,0.6030845,0.32382196,0.058272216,1.1496192,-0.07583337,-0.38381022,-0.038313106,-0.059901193,-0.009235247,0.3108128,-0.16180591,0.076915234,-0.628948,0.012136114,0.34915993,0.11169649,-0.027603326,0.12088766,0.015754415,-0.07528986,0.7361764,-0.375096,0.6838765,0.34126616,-0.17232133,0.5670233,0.1842313,-0.9191954,-0.05715491,-0.39360154,0.82914567,0.7040055,-0.9378289,-0.6692771,0.20666184,0.4659079,-0.9852135,-0.05060219,-0.8569875,-0.38273558,-1.0934033,0.09667993,0.5391056,-0.21248612,0.97338355,-0.25036633,0.13185519,-1.0454012,0.70061225,0.56089413,-0.23353964,0.3873815,0.61252654,-0.18594839,-1.1697508,-0.13116485,-0.15274367,-0.3425766,0.7229913,0.46215582,0.19172013,0.7315542,0.20280124,-0.47283942,0.11268868,0.21760689,-1.2414601,-0.54571706,0.6389718,-0.16051203,0.20270866,0.3108413,-0.417417,0.5325184,-0.30982396,0.3395837,-0.054592233,0.18700255,0.020021215,-0.44557497,0.17568253,0.066207305,-0.41227284,0.038476147,-0.56490237,-0.23229218,0.3676868,1.0247589,0.45013517,0.45407248,0.15603545,-0.111387715,0.011789778,-0.33382058,0.28754377,-0.3872725,0.79699403,-0.07860645,-0.6673928,-0.052668147,-0.15879771,0.6256208,0.028558478,-0.30958888,-0.7050788,0.22046235,-0.78366685,0.098655045,0.50495523,0.4559019,0.17166498,-0.43103623,-0.54671085,-0.65522283,-1.557738,-0.76205677,-0.35500312,-0.11766226,0.14130858,0.10890559,0.13084802,0.4764247,0.0681049,0.19539425,0.09600227,-0.304814,-0.27403235,1.1285267,0.24745204,0.33300155,0.23377757,0.005707037,0.076807685,0.6401494,-0.3869688,-0.69027025,0.2385011,0.02566953,-0.46519974,-0.08308293,-0.3190682,-0.32580185,-0.34240428,0.23345542,0.06658649,-0.1667445,0.44380632,0.08797021,0.3555495,0.9894684,0.804572,0.7345692,0.33207488,-0.17618023,-0.6734484,-0.59454125,0.6396385,-0.0023660294,-0.8215612,0.28582826,0.49408203,-0.80100435,-0.2171121,-0.737516,0.16058105,-0.572672,0.67947215,1.0157514,-0.46116358,0.14001904,0.47349715,-0.5133047,0.45741102,-0.47147244,0.14892927,0.15213883,-0.1770583,-0.14278744,-0.7972758,-0.96667296,0.25489458,-0.5524288,-0.17040165,5.393658E-5,-0.13759252,-0.44481844,0.50669485,0.47381333,-0.51452607,-0.1677315,-0.25661966,-0.060210813,-0.41531605,-0.7907455,-1.0915484,0.3523149,0.11650098,-0.9119422,1.2233778,0.5198397,0.69896245,0.8568886,-0.4719389,0.221684,-0.120144926,-0.91353774,0.15424588,0.30815247,0.898391,-0.2668036,0.6571687,-0.5245165,-0.04805845,-0.4360594,0.45182943,-0.32383764,-0.2717177,-0.029871043,-0.3980963,0.24144305,-0.12073473,-0.3863538,-0.29163685,-0.81181383,0.2518887,-0.15020402,-1.2167256,0.09825281,0.25232646,0.35458043,-0.42285445,0.5985994,0.2958827,-0.12750606,-0.2917953,0.468063

In [0]:

word2vec_train = spark.sql('''select distinct a.* from tempdb.word2vec_completeset a inner join tempdb.hadm_dataset_lookup b on a.id = b.row_id and b.group_type = 'train' ''')
word2vec_test = spark.sql('''select distinct a.* from tempdb.word2vec_completeset a inner join tempdb.hadm_dataset_lookup b on a.id = b.row_id and b.group_type = 'test' ''')
word2vec_valid = spark.sql('''select distinct a.* from tempdb.word2vec_completeset a inner join tempdb.hadm_dataset_lookup b on a.id = b.row_id and b.group_type = 'valid' ''')


In [0]:
# 'DATA_WORD2VECV1_HADM_TOP10'
# word2vec_valid.write.csv("/dbfs/FileStore/data/DATA_WORD2VECV1_HADM_TOP10_train.csv")
word2vec_valid.repartition(1).write.format('com.databricks.spark.csv').save("/dbfs/FileStore/output/DATA_WORD2VECV1_HADM_TOP10_valid.csv",header = 'true')
word2vec_train.repartition(1).write.format('com.databricks.spark.csv').save("/dbfs/FileStore/output/DATA_WORD2VECV1_HADM_TOP10_train.csv",header = 'true')
word2vec_test.repartition(1).write.format('com.databricks.spark.csv').save("/dbfs/FileStore/output/DATA_WORD2VECV1_HADM_TOP10_test.csv",header = 'true')


In [0]:
word2vec_test

In [0]:
%sql
select * 
from tempdb.word2vec_completeset
limit 10

id,4019,2724,25000,4280,41401,53081,51881,42731,5849,5990,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
55719,1,0,0,0,0,0,0,0,0,0,-1.1525245,0.15723443,0.1838233,-0.57781225,0.3511713,1.0269778,1.1865035,-0.3645481,-0.32536992,0.7053559,-0.05729557,0.30402762,-0.511216,0.3449339,-0.26530087,-0.19545084,-0.035034392,0.6030845,0.32382196,0.058272216,1.1496192,-0.07583337,-0.38381022,-0.038313106,-0.059901193,-0.009235247,0.3108128,-0.16180591,0.076915234,-0.628948,0.012136114,0.34915993,0.11169649,-0.027603326,0.12088766,0.015754415,-0.07528986,0.7361764,-0.375096,0.6838765,0.34126616,-0.17232133,0.5670233,0.1842313,-0.9191954,-0.05715491,-0.39360154,0.82914567,0.7040055,-0.9378289,-0.6692771,0.20666184,0.4659079,-0.9852135,-0.05060219,-0.8569875,-0.38273558,-1.0934033,0.09667993,0.5391056,-0.21248612,0.97338355,-0.25036633,0.13185519,-1.0454012,0.70061225,0.56089413,-0.23353964,0.3873815,0.61252654,-0.18594839,-1.1697508,-0.13116485,-0.15274367,-0.3425766,0.7229913,0.46215582,0.19172013,0.7315542,0.20280124,-0.47283942,0.11268868,0.21760689,-1.2414601,-0.54571706,0.6389718,-0.16051203,0.20270866,0.3108413,-0.417417,0.5325184,-0.30982396,0.3395837,-0.054592233,0.18700255,0.020021215,-0.44557497,0.17568253,0.066207305,-0.41227284,0.038476147,-0.56490237,-0.23229218,0.3676868,1.0247589,0.45013517,0.45407248,0.15603545,-0.111387715,0.011789778,-0.33382058,0.28754377,-0.3872725,0.79699403,-0.07860645,-0.6673928,-0.052668147,-0.15879771,0.6256208,0.028558478,-0.30958888,-0.7050788,0.22046235,-0.78366685,0.098655045,0.50495523,0.4559019,0.17166498,-0.43103623,-0.54671085,-0.65522283,-1.557738,-0.76205677,-0.35500312,-0.11766226,0.14130858,0.10890559,0.13084802,0.4764247,0.0681049,0.19539425,0.09600227,-0.304814,-0.27403235,1.1285267,0.24745204,0.33300155,0.23377757,0.005707037,0.076807685,0.6401494,-0.3869688,-0.69027025,0.2385011,0.02566953,-0.46519974,-0.08308293,-0.3190682,-0.32580185,-0.34240428,0.23345542,0.06658649,-0.1667445,0.44380632,0.08797021,0.3555495,0.9894684,0.804572,0.7345692,0.33207488,-0.17618023,-0.6734484,-0.59454125,0.6396385,-0.0023660294,-0.8215612,0.28582826,0.49408203,-0.80100435,-0.2171121,-0.737516,0.16058105,-0.572672,0.67947215,1.0157514,-0.46116358,0.14001904,0.47349715,-0.5133047,0.45741102,-0.47147244,0.14892927,0.15213883,-0.1770583,-0.14278744,-0.7972758,-0.96667296,0.25489458,-0.5524288,-0.17040165,5.393658E-5,-0.13759252,-0.44481844,0.50669485,0.47381333,-0.51452607,-0.1677315,-0.25661966,-0.060210813,-0.41531605,-0.7907455,-1.0915484,0.3523149,0.11650098,-0.9119422,1.2233778,0.5198397,0.69896245,0.8568886,-0.4719389,0.221684,-0.120144926,-0.91353774,0.15424588,0.30815247,0.898391,-0.2668036,0.6571687,-0.5245165,-0.04805845,-0.4360594,0.45182943,-0.32383764,-0.2717177,-0.029871043,-0.3980963,0.24144305,-0.12073473,-0.3863538,-0.29163685,-0.81181383,0.2518887,-0.15020402,-1.2167256,0.09825281,0.25232646,0.35458043,-0.42285445,0.5985994,0.2958827,-0.12750606,-0.2917953,0.468063

In [0]:
# saves the word2vec model to be used later.
#

# open a saved word2vec model 
#import gensim
#model_w2v=gensim.models.Word2Vec.load('./model_word2vec')


#model_w2v.wv.save_word2vec_format('./model_word2vec_v2_300dim.txt', binary=False)